In [6]:
def get_open_spaces(s):
    open_spaces = []
    for i in range(len(s)):
        for j in range(len(s[i])):
            print(s[i][j], s[i][j]==0)
            if (s[i][j] == 0):
                open_spaces.append((i,j))
    return open_spaces

def get_next_states(state,move):
    res = [] 
    s  = copy.deepcopy(state)
    i,j = move
    s[i][j] = 1
    if is_winner(s):
        res.append([s, 1])
        return res
    else:
        oppo_open_spaces = get_open_spaces(s)
        prob = 1 / len(oppo_open_spaces)
        for space in oppo_open_spaces:
            oppo_state = copy.deepcopy(s)
            i,j = space
            oppo_state[i][j] = 1
            res.append([oppo_state, prob])
    return res

def create_all_states(curr_state, all_states, V):
    curr_state = np.array(curr_state)
    state_string = np.array2string(curr_state)
    V[state_string] = [0, curr_state]
    all_states.append(curr_state)
    spaces = get_open_spaces(curr_state)
    if len(spaces) == 0:
        return all_states

    for space in spaces:
        
        new_state = copy.deepcopy(curr_state)
        print("first",curr_state)
        i,j = space
        new_state[i][j] = 1
        print('i,j,final',i, j, new_state)
        
        if is_winner(new_state):
            V[np.array2string(new_state)] = [0, new_state]
            all_states.append(new_state)
            continue
        else:
            oppo_spaces = get_open_spaces(new_state)
            for oppo_space in oppo_spaces:
                oppo_state = list(copy.deepcopy(new_state))
                i,j = oppo_space
                oppo_state[i][j] = -1
                all_states = create_all_states(oppo_spaces, all_states, V)
    return all_states

def value_interation(state, tolerant, string_state):

    allStates = []
    new_V = {}
    create_all_states(state, allStates, new_V)
    num_iteration = 1
    graph = [0]
    opt_action = []

    while True:
        V = copy.deepcopy(new_V)
        err = 0
        num_iteration += 1
        for state_string in V:
            curr_state = V[state_string][1]
            
            open_spaces = get_open_spaces(curr_state)
            if not open_spaces:
                new_V[state_string] = [get_reward(curr_state,open_spaces,1), curr_state]
            else: 
                score_for_actions = [0] * len(open_spaces)
                for i, move in enumerate(open_spaces):
                    nextStatesAndProbs = get_next_states(curr_state, move)
                    for nextStateAndProb in nextStatesAndProbs:
                        score_for_actions[i] += nextStateAndProb[1] * V[np.array2string(nextStateAndProb[0])][0]
                new_V[state_string][0] = get_reward(curr_state,open_spaces,1) + gamma * max(score_for_actions)
            if state_string == string_state:
                graph.append(new_V[state_string][0])
                opt_action  = open_spaces[score_for_actions.index(max(score_for_actions))]
            if abs(new_V[state_string][0] - V[state_string][0]) > err:
                err = abs(new_V[state_string][0] - V[state_string][0])
        if err < tolerant:

            return opt_action


In [8]:
import numpy as np
import copy
target = np.array([[-1,0,-1], [1,1,0], [ -1,0, 0]])

print(create_all_states(target,[],{}))

-1 False
0 True
-1 False
1 False
1 False
0 True
-1 False
0 True
0 True
first [[-1  0 -1]
 [ 1  1  0]
 [-1  0  0]]
i,j,final 0 1 [[-1  1 -1]
 [ 1  1  0]
 [-1  0  0]]


NameError: name 'is_winner' is not defined